## Riduzione del dataset

In questo notebook, eseguiremo la riduzione del dataset. L'obbiettivo è quello di diminuire la quantità di audio per label ma mantenendo le caratteristiche di sbilanciamento generali del dataset.

# Installazione e importazione delle librerie necessarie

Installiamo e importiamo le librerie che ci torneranno utili per la riduzione del dataset. In questo caso Tensorflow sarà utilizzato per estrarre il dataset dalla directory, mentre le restanti saranno utilizate principalmente per tutto ciò che comprende la riduzione del dataset

In [ ]:
# Installiamo la libreria di tensorflow
!pip install tensorflow

In [1]:
import tensorflow as tf
import os
import random
import shutil

# Estrazione dataset
La funzione `audio_dataset_from_directory(directory)` è un modo semplice e veloce ottenere per ottenere informazioni dalle **sottodirectory di un dataset**. Il funzionamento di base è uguale a altre funzioni viste durante il corso, come per esempio `image_dataset_from_directory`. Nel nostro caso il dataset è composto da **30 sottodirectory**, che corrispondono a **30 classi**, che verranno rappresentate come **indice numerico**, e ha una **batch_size di 32**.

In [2]:
# estraiamo il nostro dataset originale
train_ds = tf.keras.utils.audio_dataset_from_directory(directory='../original/train/audio')

Found 64721 files belonging to 30 classes.


In [3]:
# stampiamo il numero delle classi e i nomi delle classi del dataset
print('Numero classi:', len(train_ds.class_names))
print('Classi:',train_ds.class_names)

Numero classi: 30
Classi: ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


# Funzioni utilizzate
Andiamo a creare delle funzioni che ci serviranno per la riduzione del dataset.

In questo caso la prima funzione `copy_files` si occupa di creare la directory di destinazione (quindi quella del dataset ridotto) e di copiare i file scelti. La seconda funzione `downsample_class` riduce il dataset, mantenendo in questo caso un quantitativo di audio proporzionato alla quantità iniziale.

Inizialmente avevamo lasciato 300 audio per ogni label, ma abbiamo pensato che mantenere una percentuale invece di un numero fisso mantiene le caratteristiche di sbilanciamento del dataset.

In [4]:
# funzione per copiare i file
def copy_files(files, src_dir, dest_dir):
    if not os.path.exists(dest_dir): # se la directory di destinazione non esiste allora la creo
        os.makedirs(dest_dir)
    for file in files: # copio ogni file che mi viene passato nella directory di destinazione, quindi nella directory della classe
        shutil.copy(os.path.join(src_dir, file), os.path.join(dest_dir, file))

# funzione per diminuire la quantità di audio per label
def downsample_class(class_dir, class_name, target_dir, ratio):
    files = os.listdir(class_dir) # mi salvo tutti i file della classe in una variabile
    num_files_to_keep = int(len(files) * ratio) # calcolo la quantità di file che devo mantenere per la classe in questione
    files_to_keep = random.sample(files, num_files_to_keep) # scelgo randomicamente i file da mantenere
    
    # chiamo la funzione precedentemente dichiarata per copiare i file nella directory
    copy_files(files_to_keep, class_dir, os.path.join(target_dir, class_name))

Quindi utilizziamo le funzioni appena dichiarate per ridurre il dataset dell'80%.

In [5]:
original_dir = '../original/train/audio' # path dataset originale
downsampled_dir = '../reduced_dataset/train/audio' # path dataset ridotto

classes = [dir for dir in os.listdir(original_dir) if os.path.isdir(os.path.join(original_dir, dir))]

for class_name in classes:
    class_dir = os.path.join(original_dir, class_name)
    downsample_class(class_dir, class_name, downsampled_dir, 0.20)

Con queste righe di codice abbiamo terminato la riduzione della parte di training del dataset, passando da una **dimensione iniziale di 1.90 GB con 64721 audio totali** a una **dimensione finale di 389 MB con 12933 audio totali**.